In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load all data
dataset = pd.read_csv('../input/household-power-consumption/household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])
# summarize
print(dataset.shape)
print(dataset.head())
dataset.tail()



In [ ]:
# mark all missing values
dataset.replace('?',np.nan, inplace=True)

In [ ]:
dataset.tail(365)

In [ ]:
dataset.describe(include='all')

In [ ]:
columns = list(dataset.columns)

for col in columns:
    dataset[col] = dataset[col].astype(str).astype('float32')

In [ ]:
dataset.dtypes

In [ ]:

dataset.fillna(dataset.mean(),inplace=True)

In [ ]:
dataset.isnull().sum()

In [ ]:
import matplotlib.pyplot as plt

df_daily = dataset.resample('D').sum()

In [ ]:

plt.plot(df_daily.index,df_daily.Global_active_power)


In [ ]:


dataset.corr()

In [ ]:


df_daily.reset_index(inplace=True)


x_test = df_daily.tail(365)
x_train = df_daily[:-365]

df_daily.dtypes

In [ ]:


df_uni = df_daily[['datetime','Global_active_power']]

# making a new dataframe from df_daily2 but renaming the columns
new_df = df_uni.rename(columns={"datetime": "ds", "Global_active_power": "y"})



In [ ]:
from fbprophet import Prophet
x_test_uni = new_df.tail(365)
x_train_uni = new_df[:-365]

In [ ]:
model = Prophet()

In [ ]:
model.fit(x_train_uni)

In [ ]:
future = model.make_future_dataframe(periods=365 , freq= 'D' )
result_uni = model.predict(future)


In [ ]:
result_uni.shape

In [ ]:
result_uni.head()

In [ ]:
model.plot(result_uni)
plt.show()



In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
# MAPE
mean_absolute_percentage_error(x_test_uni['y'], result_uni['yhat'][-365:])

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(x_test_uni['y'], result_uni['yhat'][-365:]))
print(rmse)

In [ ]:
# statsmodels is a Python module for statistical models,conducting statistical tests and statistical data exploration.
import statsmodels.api as sm

from pylab import rcParams
#pylab is a module within the matplotlib library that was built to mimic MATLAB’s global style , All module objects starting with “rc” are a means to interact with your plot styles and settings

rcParams['figure.figsize']= 15,8

In [ ]:


plt.plot(result_uni['ds'],result_uni['yearly'])
plt.show()

In [ ]:
model.plot_components(result_uni)

In [ ]:


new_df = df_daily.copy()



# making a new dataframe from df_daily2 but renaming the columns
new_df_mul= new_df.rename(columns={"datetime": "ds", "Global_active_power": "y","Global_reactive_power": "add1","Voltage": "add2","Global_intensity": "add3","Sub_metering_1": "add4","Sub_metering_2": "add5","Sub_metering_3": "add6"})


In [ ]:


x_test_multi = new_df_mul.tail(365)
x_train_multi = new_df_mul[:-365]

In [ ]:
print(x_train_multi)

In [ ]:
# regressors = ['add1','add2','add3','add4','add5','add6']
model_multi = Prophet()


In [ ]:
model_multi.fit(x_train_multi)

In [ ]:
future = model_multi.make_future_dataframe(periods=365,freq='D')
result_multi= model_multi.predict(future)

In [ ]:
model_multi.plot(result_multi)
plt.show()



In [ ]:
# MAPE
mean_absolute_percentage_error(x_test_multi['y'], result_multi['yhat'][-365:])

In [ ]:
rmse_multi = sqrt(mean_squared_error(x_test_multi['y'], result_multi['yhat'][-365:]))
print(rmse_multi)

In [ ]:
print(x_train_uni)

In [ ]:
print(x_train_multi)

In [ ]:
model_multi.plot_components(result_multi)